In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



In [4]:
pip install kaggle


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
#IMPORTING DEPENDENCY


In [6]:
import numpy as np
import pandas as pd
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #used to reduce words to their root or base form
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
twitter_data = pd.read_csv("datasets/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1")

In [9]:
twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head()[:10]

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#naming above columns
column_names= ["target","id","date","flag","user","text"]
twitter_data = pd.read_csv("datasets/training.1600000.processed.noemoticon.csv",names=column_names, encoding="ISO-8859-1")

In [12]:
twitter_data.head()[:10]

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
#checkig missing values
twitter_data.isnull()

,target,id,date,flag,user,text
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
1599995,False,False,False,False,False,False
1599996,False,False,False,False,False,False
1599997,False,False,False,False,False,False
1599998,False,False,False,False,False,False


In [14]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
#checking distribution of datasets
twitter_data["target"].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [16]:
twitter_data["text"].value_counts()

text
isPlayer Has Died! Sorry                                                                              210
good morning                                                                                          118
headache                                                                                              115
Good morning                                                                                          112
Headache                                                                                              106
                                                                                                     ... 
braces  tell me it will be okay...                                                                      1
is stuck at home without curry                                                                          1
@mrsduryee I've applied to about 70 since I lost my job in March...it certainly FEELS like a lot!       1
The cheese I got @SarawithanR lost its sq

In [17]:
twitter_data.replace({"target": {4: 1}}, inplace=True)


In [18]:
twitter_data["target"].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [19]:

#Stemming 

In [20]:
port_stem= PorterStemmer()

In [21]:
import re
from nltk.stem import PorterStemmer

def stemming(content):
    # Using parentheses instead of square brackets for re.sub
    stemmed_content = re.sub("[^a-zA-Z]", " ", content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    
    ps = PorterStemmer()
    stemmed_content = [ps.stem(word) for word in stemmed_content]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content



In [22]:
twitter_data["stemmed_content"]=twitter_data["text"].apply(stemming) #take 50 minutes

In [23]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com y zl awww that s a...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can t updat hi facebook by te...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dive mani time for the ball manag t...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass no it s not behav at all i m m...


In [24]:
#separating data and label

In [25]:
X= twitter_data["stemmed_content"].values
Y= twitter_data["target"].values

In [26]:
print(X)
print(Y)

['switchfoot http twitpic com y zl awww that s a bummer you shoulda got david carr of third day to do it d'
 'is upset that he can t updat hi facebook by text it and might cri as a result school today also blah'
 'kenichan i dive mani time for the ball manag to save the rest go out of bound'
 ... 'are you readi for your mojo makeov ask me for detail'
 'happi th birthday to my boo of alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


In [27]:
#Spillitng training and testing data

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2) #usimg stratigy= y to equally spliting during training

In [29]:
X_train.shape

(1280000,)

In [30]:
X_test.shape


(320000,)

In [31]:
Y_train.shape

(1280000,)

In [32]:
Y_test.shape

(320000,)

In [33]:
print(X_train)

['about to watch saw iv and drink a lil wine' 'hatermagazin i m in'
 'even though it my favourit drink i think it the vodka and coke that wipe my mind all the time think im gonna have to find a new drink'
 ... 'is eager for monday afternoon'
 'hope everyon and their mother had a great day can t wait to hear what the guy have in store tomorrow'
 'i love wake up to folger too bad my voic wa deeper than hi']


In [34]:
print(X_test)

['mmangen m do fine i haven t had much time to chat on twitter hubbi is back for the summer amp tend to domin my free time'
 'at ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb it wa onli a bay area thang dammit' ...
 'destini nevertheless hooray for member and have a wonder and safe trip'
 'not feel too well' 'supersandro thank you']


In [35]:
#using vectorizer to convert text to vector

In [36]:
vectorizer= TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [37]:
X_test = vectorizer.transform(X_test)

In [38]:
print(X_train)

  (0, 443091)	0.42499435013319353
  (0, 235060)	0.3979796718985349
  (0, 109311)	0.35571727528522945
  (0, 16222)	0.15448255989577533
  (0, 185208)	0.5001351520889395
  (0, 354563)	0.3400227113489447
  (0, 436736)	0.25832609761141173
  (0, 411299)	0.12543227444715074
  (0, 1960)	0.24975207525435875
  (1, 179244)	0.21530681081326286
  (1, 160645)	0.9765464542034966
  (2, 288488)	0.15525132814420295
  (2, 132317)	0.18764608140093672
  (2, 160854)	0.11128349809796605
  (2, 150722)	0.17390430347392352
  (2, 178072)	0.14973147343115414
  (2, 409165)	0.14029059989672196
  (2, 11253)	0.13268982365918122
  (2, 266745)	0.22309971079942176
  (2, 443455)	0.30968970461207745
  (2, 400872)	0.10919973617378233
  (2, 77931)	0.2893256489481644
  (2, 433582)	0.30488022171033813
  (2, 401239)	0.15421558405984295
  (2, 406421)	0.2969220344399517
  :	:
  (1279998, 124771)	0.2684571615089851
  (1279998, 439913)	0.20630210124846357
  (1279998, 169472)	0.2214801864717222
  (1279998, 157316)	0.218688343833733

In [39]:
print(X_test)

  (0, 421006)	0.1641136097461986
  (0, 411299)	0.15363735451176994
  (0, 409165)	0.2879144949200723
  (0, 401239)	0.07912308812222692
  (0, 398928)	0.3197922634909907
  (0, 388370)	0.20139125990222212
  (0, 301865)	0.11308352156103443
  (0, 281182)	0.0932833224429557
  (0, 279098)	0.1632851061506343
  (0, 271032)	0.41548310342582123
  (0, 183023)	0.10169980247255428
  (0, 171389)	0.2570229272802928
  (0, 160881)	0.21333363690176793
  (0, 157316)	0.15472633610001849
  (0, 138171)	0.21699333713272062
  (0, 136132)	0.10482875922035323
  (0, 132370)	0.23382272426688716
  (0, 106074)	0.3348611626295079
  (0, 105210)	0.13689296457063335
  (0, 67830)	0.24550114468346762
  (0, 31169)	0.14883503848400464
  (0, 15110)	0.1574989475067553
  (1, 366225)	0.2424922761659254
  (1, 348155)	0.4672544898941835
  (1, 256792)	0.2834672915539333
  :	:
  (319995, 155500)	0.2529438353945605
  (319995, 109384)	0.27578658016178886
  (319995, 107873)	0.304912547974264
  (319996, 455986)	0.1683799899431786
  (319

In [40]:
#Logistic Regression


In [41]:
Model= LogisticRegression(max_iter=1000)
Model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [44]:
#Accuracy Testing for model

In [45]:
X_train_predict= Model.predict(X_train)
Log_training_data_acc= accuracy_score(Y_train,X_train_predict)

In [46]:
print(Log_training_data_acc)

0.7977828125


In [56]:
#for test data 

In [47]:
Model= LogisticRegression(max_iter=1000)
Model.fit(X_test,Y_test)

LogisticRegression(max_iter=1000)

In [48]:
X_test_predict= Model.predict(X_test)
Log_test_data_acc= accuracy_score(Y_test,X_test_predict)

In [49]:
print(Log_test_data_acc)

0.815090625


In [51]:
#saving the training model
import pickle
filename= "trainedModel.sav"
pickle.dump(Model, open(filename, "wb"))

In [52]:
#Loading model for future prediction

In [53]:
loaded_model =pickle.load(open("model/trainedModel.sav", "rb"))

In [54]:
X_new= X_test[1000]
print(Y_test[1000])

prediction = Model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print("Negative Tweet" )
else:
    print("Postive Tweet")

0
[0]
Negative Tweet


In [57]:
X_new= X_test[6]
print(Y_test[6])

prediction = Model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print("Negative Tweet" )
else:
    print("Postive Tweet")

1
[1]
Postive Tweet
